# seq2seq with Fairseq

This notebook uses Fairseq and PyTorch to train a sequence-to-sequence model.

Note you must turn on GPU to use Fairseq!

> *Edit > Notebook settings > Hardware accelerator: GPU*




## Requirements

In [2]:
%cd /content/
!rm -rf fairseq
!git clone https://github.com/deeplanguageclass/fairseq.git
%cd fairseq
!ls
!pip install -r requirements.txt

/content
Cloning into 'fairseq'...
remote: Counting objects: 2189, done.
remote: Total 2189 (delta 0), reused 0 (delta 0), pack-reused 2189
Receiving objects: 100% (2189/2189), 2.75 MiB | 32.76 MiB/s, done.
Resolving deltas: 100% (1613/1613), done.
/content/fairseq
CONTRIBUTING.md       fairseq.gif		PATENTS		  scripts
distributed_train.py  generate.py		preprocess.py	  setup.py
eval_lm.py	      interactive.py		README.md	  tests
examples	      LICENSE			requirements.txt  train.py
fairseq		      multiprocessing_train.py	score.py
    100% |████████████████████████████████| 430kB 9.4MB/s 
    100% |████████████████████████████████| 519.5MB 29kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x58522000 @  0x7fe8c2ff21c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100%

In [3]:
!python setup.py build
!python setup.py develop

running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/scripts
copying scripts/__init__.py -> build/lib.linux-x86_64-3.6/scripts
copying scripts/average_checkpoints.py -> build/lib.linux-x86_64-3.6/scripts
copying scripts/build_sym_alignment.py -> build/lib.linux-x86_64-3.6/scripts
creating build/lib.linux-x86_64-3.6/fairseq
copying fairseq/progress_bar.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/utils.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/multiprocessing_pdb.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/__init__.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/tokenizer.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/trainer.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/meters.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/bleu.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/fp16_trainer.py -> build/lib.linux-x86_64-3.6/fairseq
copyin

## Data pre-processing

In [0]:
%cd examples/translation/
!bash prepare-iwslt14.sh
%cd ../..

!python preprocess.py --source-lang de --target-lang en \
  --trainpref examples/translation/iwslt14.tokenized.de-en/train \
  --validpref examples/translation/iwslt14.tokenized.de-en/valid \
  --testpref examples/translation/iwslt14.tokenized.de-en/test \
  --destdir data-bin/iwslt14.tokenized.de-en

/content/fairseq/examples/translation
Cloning Moses github repository (for tokenization scripts)...
Cloning into 'mosesdecoder'...
remote: Counting objects: 147104, done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 147104 (delta 0), reused 2 (delta 0), pack-reused 147098
Receiving objects: 100% (147104/147104), 129.65 MiB | 21.17 MiB/s, done.
Resolving deltas: 100% (113696/113696), done.
Cloning Subword NMT repository (for BPE pre-processing)...
Cloning into 'subword-nmt'...
remote: Counting objects: 455, done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 455 (delta 18), reused 18 (delta 9), pack-reused 420
Receiving objects: 100% (455/455), 204.70 KiB | 12.79 MiB/s, done.
Resolving deltas: 100% (262/262), done.
--2018-08-16 11:18:05--  https://wit3.fbk.eu/archive/2014-01/texts/de/en/de-en.tgz
Resolving wit3.fbk.eu (wit3.fbk.eu)... 217.77.80.8
Connecting to wit3.fbk.eu (wit3.fbk.eu)|217.77.80.8|:443... connected.
HTTP request sent, awaiting response..

## Training

In [1]:
!mkdir -p checkpoints/fconv
!CUDA_VISIBLE_DEVICES=0 python train.py data-bin/iwslt14.tokenized.de-en \
  --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
  --arch fconv_iwslt_de_en --save-dir checkpoints/fconv --skip-invalid-size-inputs-valid-test

python3: can't open file 'train.py': [Errno 2] No such file or directory


## Testing

In [0]:
!python generate.py data-bin/iwslt14.tokenized.de-en \
  --path checkpoints/fconv/checkpoint_best.pt \
  --batch-size 128 --beam 5